In [1]:
import cv2
import numpy as np

### Yolo 로드

In [26]:
# # weights, cfg파일을 불러와서 yolo의 네트워크와 연결
net = cv2.dnn.readNet('./data/yolov3.weights', './data/yolov3.cfg')

#### 내용 정리

- yolov3.weights : yolov3의 훈련된 가중치를 저장하고 있는 이진 파일
- yolov3.cfg :  yolov3의 네트워크 구성을 저장하고 있는 텍스트 파일

In [27]:
classes = []   # class 배열
with open('./data/coco.names', 'r') as f:  # coco 파일 로드
    classes = [line.strip() for line in f.readlines()] # 읽어온 coco 파일을 whitespace(공백라인)를 제거하여 classes 배열에 할당

# net.getLayerNames() - 출력 레이어를 가져옴
layer_names = net.getLayerNames() # 네트워크의 모든 레이어 이름을 가져와서 layer_names에 할당
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()] # 레이어 중 출력 레이어의 인덱스를 가져와서 output_layers에 할당

colors = np.random.uniform(0, 255, size=(len(classes), 3)) # 클래스의 갯수만큼 랜덤으로 BRG 배열을 생성

### 이미지 불러오기

In [43]:
img = cv2.imread('../images/cats-and-dogs.jpg')
img = cv2.resize(img, None, fx=0.4, fy=0.4)  # 이미지 크기를 재설정 

height, width, channel = img.shape  # 해당 이미지의 속성

### 물체 감지

In [29]:
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)  # 이미지를 blob(Binary Large Object) 객체로 처리
net.setInput(blob)  # blob 객체에 setInput 함수 적용 / 네트워크에 blob을 입력
outs = net.forward(output_layers) # output_layers를 네트워크 순방향으로 실행(추론)

### 물체 인식 정보 저장

In [ ]:
class_ids = []  # 인식한 사물 클래스 아이디
confidences = [] # 0에서 1까지 사물 인식에 대한 신뢰도
boxes = [] # 사물을 인식해서 그릴 상자에 대한 정보

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores) # scores 중에서 최대값을 색인하여 class_id에 할당
        confidence = scores[class_id] # scores 중에서 class_id에 해당하는 값을 confidence에 할당

        if confidence > 0.5: # 사물이 인식되었다고 판단
            # 객체 탐지(Object detected)
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # 좌표
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

### 노이즈 제거
- 같은 사물에 대해서 박스가 여러 개인 것을 제거하는 Non Maximum Suppresion 작업 수행

In [24]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

### 결과 이미지 출력

In [46]:
font = cv2.FONT_HERSHEY_PLAIN  # Font종류 중 하나인 FONT_HERSHEY_PLAIN(작은 크기 산세리프 폰트)를 적용

for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])  # 클래스 아이디 지정해둔 것을 label변수에 저장
        color = colors[i]  # colors 배열에 색상을 넣어둔 것을 color에 저장
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2) # 사각형
        cv2.putText(img, label, (x, y-5), font, 1, color, 2) # yolo에서 학습된 사물의 명칭을 쓰기기

cv2.imshow('image', img) # 이미지 출력
cv2.waitKey()  # waitKey() : 0 -> 키 입력이 있을 때까지 무한 대기

# save output
cv2.imwrite('./output/yolov3_detection.png', img)
 
cv2.destroyAllWindows() # 열린 모든 창을 파괴(!)